In [5]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType,StringType, FloatType
import pyspark
from pyspark.sql import SparkSession
import logging

In [12]:
spark = (
    SparkSession.builder.appName("Read_Parquet_File")
    .config("spark.jars.packages",
            "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.0,"
            "software.amazon.awssdk:bundle:2.24.8,"
            "software.amazon.awssdk:url-connection-client:2.24.8,"
            "org.apache.hadoop:hadoop-aws:3.3.4,"
            "com.amazonaws:aws-java-sdk-bundle:1.12.375")
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config("spark.sql.catalog.lakehouse_prod_catalog", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.lakehouse_prod_catalog.type", "nessie")  # <--- use nessie, not rest
    .config("spark.sql.catalog.lakehouse_prod_catalog.uri", "http://nessie:19120/api/v2")  # <--- correct URL
    .config("spark.sql.catalog.lakehouse_prod_catalog.warehouse", "s3://warehouse/")
    .config("spark.sql.catalog.lakehouse_prod_catalog.ref", "main")  # <--- default Nessie branch
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")   # MinIO endpoint
    .config("spark.hadoop.fs.s3a.access.key", "admin")            # MinIO access key
    .config("spark.hadoop.fs.s3a.secret.key", "password")            # MinIO secret key
    .config("spark.hadoop.fs.s3a.path.style.access", "true")           # important for MinIO
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .getOrCreate()
)

In [20]:
spark = (
    SparkSession.builder.appName("Read_Parquet_File")
    .config("spark.jars.packages",
            ",".join([
                "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.0",
                "org.apache.hadoop:hadoop-aws:3.3.4",
                "com.amazonaws:aws-java-sdk-bundle:1.12.375"
            ]))
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
    .config("spark.hadoop.fs.s3a.access.key", "admin")
    .config("spark.hadoop.fs.s3a.secret.key", "password")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .getOrCreate()
)

In [18]:
def extract_csv_from_lake_s3(bucket: str, folder: str, filename: str):
    """
    Read a CSV file from MinIO (S3-compatible storage) into a Spark DataFrame.
    """

    path = f"s3a://{bucket}/{folder}/{filename}"

    logging.info(f"Reading file from MinIO using Spark: {path}")

    try:
        df = (
            spark.read
            .option("header", "true")
            .option("inferSchema", "true")
            .csv(path)
        )

        logging.info(f"Successfully extracted {filename} with {df.count()} rows.")
        df.show(5, truncate=False)
        return df

    except Exception as e:
        logging.error(f"Error reading from MinIO: {e}")
        raise

In [22]:
bucket= 'landing-data'
folder= 'HR-LAKE-DATASET'
filename= 'HRIS_EMPLOYEE_20250810.csv'

df = extract_csv_from_lake_s3(bucket, folder, filename)
df

25/10/11 17:02:55 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: s3a://landing-data/HR-LAKE-DATASET/HRIS_EMPLOYEE_20250810.csv.
java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:53)
	at org.apache.spark.sql.execution.dat

Py4JJavaError: An error occurred while calling o202.csv.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:724)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(Da

In [1]:
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 4.3 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [12]:
import boto3
import pandas as pd
from io import StringIO

# S3 Credential
aws_access_key_id = "admin"
aws_secret_access_key = "password"

# Create S3 Client for MinIO
s3 = boto3.client(
    "s3",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    endpoint_url="http://minio:9000",  # MinIO endpoint
    use_ssl=False
)

# Bucket Name
bucket_name = "landing-data"

# List objects in bucket
response = s3.list_objects_v2(Bucket=bucket_name)
objects_list = response.get("Contents", [])

# Loop through objects and read CSV into Pandas DataFrame
dfs = []  # list to store DataFrames
for obj in objects_list:
    obj_name = obj["Key"]
    response = s3.get_object(Bucket=bucket_name, Key=obj_name)
    object_content = response["Body"].read().decode("utf-8")

    # Convert CSV text to Pandas DataFrame
    df = pd.read_csv(StringIO(object_content))
    dfs.append(df)

# Combine all files into a single DataFrame (optional)
final_df = pd.concat(dfs, ignore_index=True)

final_df.head()

,Unnamed: 0,FirstName,LastName,StartDate,ExitDate,Title,Supervisor,ADEmail,BusinessUnit,EmployeeStatus,...,Satisfaction Score,Work-Life Balance Score,Training Date,Training Program Name,Training Type,Training Outcome,Location,Trainer,Training Duration(Days),Training Cost
0,0,Uriah,Bridges,20-Sep-19,NaN,Production Technician I,Peter Oneill,uriah.bridges@bilearner.com,CCDR,Active,...,2,3,15-Jul-23,Leadership Development,Internal,Failed,South Marisa,Taylor Rodriguez,2,606.11
1,1,Paula,Small,11-Feb-23,NaN,Production Technician I,Renee Mccormick,paula.small@bilearner.com,EW,Active,...,1,5,12-Sep-22,Customer Service,External,Incomplete,Tammieville,Kelly Patterson DDS,4,673.02
2,2,Edward,Buck,10-Dec-18,NaN,Area Sales Manager,Crystal Walker,edward.buck@bilearner.com,PL,Active,...,2,1,13-Aug-22,Leadership Development,External,Failed,East Roberthaven,Taylor Thomas,2,413.28
3,3,Michael,Riordan,21-Jun-21,NaN,Area Sales Manager,Rebekah Wright,michael.riordan@bilearner.com,CCDR,Active,...,5,4,15-Dec-22,Project Management,External,Completed,Garzatown,Holly Elliott,3,663.78
4,4,Jasmine,Onque,29-Jun-19,NaN,Area Sales Manager,Jason Kim,jasmine.onque@bilearner.com,TNS,Active,...,5,3,13-Jul-23,Technical Skills,External,Failed,Lake Meganville,Donald Martinez,5,399.03


In [21]:
import boto3
import pandas as pd
from io import StringIO

aws_access_key_id = "admin"
aws_secret_access_key = "password"

bucket = "landing-data"
folder = "HR-LAKE-DATASET"
file = "HRIS_EMPLOYEE_20250810.csv"

key = f"{folder}/{file}"

s3 = boto3.client(
    "s3",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    endpoint_url="http://minio:9000",
    use_ssl=False
)

response = s3.get_object(Bucket=bucket, Key=key)
data = response["Body"].read().decode("utf-8")

df = pd.read_csv(StringIO(data))
df.head()


,Unnamed: 0,FirstName,LastName,StartDate,ExitDate,Title,Supervisor,ADEmail,BusinessUnit,EmployeeStatus,...,Satisfaction Score,Work-Life Balance Score,Training Date,Training Program Name,Training Type,Training Outcome,Location,Trainer,Training Duration(Days),Training Cost
0,0,Uriah,Bridges,20-Sep-19,NaN,Production Technician I,Peter Oneill,uriah.bridges@bilearner.com,CCDR,Active,...,2,3,15-Jul-23,Leadership Development,Internal,Failed,South Marisa,Taylor Rodriguez,2,606.11
1,1,Paula,Small,11-Feb-23,NaN,Production Technician I,Renee Mccormick,paula.small@bilearner.com,EW,Active,...,1,5,12-Sep-22,Customer Service,External,Incomplete,Tammieville,Kelly Patterson DDS,4,673.02
2,2,Edward,Buck,10-Dec-18,NaN,Area Sales Manager,Crystal Walker,edward.buck@bilearner.com,PL,Active,...,2,1,13-Aug-22,Leadership Development,External,Failed,East Roberthaven,Taylor Thomas,2,413.28
3,3,Michael,Riordan,21-Jun-21,NaN,Area Sales Manager,Rebekah Wright,michael.riordan@bilearner.com,CCDR,Active,...,5,4,15-Dec-22,Project Management,External,Completed,Garzatown,Holly Elliott,3,663.78
4,4,Jasmine,Onque,29-Jun-19,NaN,Area Sales Manager,Jason Kim,jasmine.onque@bilearner.com,TNS,Active,...,5,3,13-Jul-23,Technical Skills,External,Failed,Lake Meganville,Donald Martinez,5,399.03


In [23]:
spark.stop()

In [29]:
from datetime import datetime

now = datetime.now()
f = now.strftime("%Y-%m-%d %H:%M:%S")
print(f)

2025-10-11 17:17:46


In [31]:
from datetime import datetime
import pytz

# The standard IANA timezone identifier for Thailand
thai_timezone_name = 'Asia/Bangkok'
thai_tz = pytz.timezone(thai_timezone_name)

# Get the current time in the specified timezone
now_thai_aware = datetime.now(thai_tz)

# Format the datetime object, including the timezone name (%Z)
# Note: pytz timezones often return the full identifier (e.g., 'Asia/Bangkok') for %Z, 
# or a standard abbreviation like 'ICT' depending on the Python version/system configuration.
f_thai = now_thai_aware.strftime("%Y-%m-%d %H:%M:%S")
print(f_thai)

# You can also get the timezone name/abbreviation directly from the datetime object:
print(f"Timezone Name (from tzinfo): {now_thai_aware.tzname()}") 
print(f"Timezone Offset (from utcoffset): {now_thai_aware.utcoffset()}")

2025-10-12 00:22:12
Timezone Name (from tzinfo): +07
Timezone Offset (from utcoffset): 7:00:00


In [30]:
# Import the datetime class from datetime module
from datetime import datetime

# Get the current date and time
current_datetime = datetime.now()

# Display the complete datetime object
print("Current date and time:", current_datetime)

# Access individual components
print("Year:", current_datetime.year)
print("Month:", current_datetime.month)
print("Day:", current_datetime.day)
print("Hour:", current_datetime.hour)
print("Minute:", current_datetime.minute)
print("Second:", current_datetime.second)


Current date and time: 2025-10-11 17:18:22.633017
Year: 2025
Month: 10
Day: 11
Hour: 17
Minute: 18
Second: 22
